<a href="https://colab.research.google.com/github/Shiveshrane/Research_paper_implementations/blob/main/GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decoder-only transformer

In [ ]:
import tensorflow as tf
import numpy as np
import os

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/ng-video-lecture/refs/heads/master/input.txt

--2025-01-23 13:42:53--  https://raw.githubusercontent.com/karpathy/ng-video-lecture/refs/heads/master/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-01-23 13:42:53 (26.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r') as f:
  text=f.read()

In [ ]:
print(len(text)) ## Length of text

1115394


In [ ]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[stoi[c] for c in s]
decode=lambda l:''.join([itos[i] for i in l])


print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
## Convert entire data into a tensor
data=tf.convert_to_tensor(encode(text), dtype=tf.int64)
print(data.shape, data.dtype)
print(data[:1000])

(1115394,) <dtype: 'int64'>
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1 39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39
 58 46 43 56  1 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47
 57 46 12  0  0 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53
 50 60 43 42  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47
 56 57 58  6  1 63 53 59  1 49 52 53 61  1 15 39 47 59 57  1 25 39 56 41
 47 59 57  1 47 57  1 41 46 47 43 44  1 43 52 43 51 63  1 58 53  1 58 46
 43  1 54 43 53 54 50 43  8  0  0 13 50 50 10  0 35 43  1 49 52 53 61  5
 58  6  1 61 43  1 49 52 53 61  5 58  8  0  0 18 47 56 57 58  1 15 47 58
 47 64 43 52 10  0 24 43 58  1 59 57  1 49 47 50 50  1 46 47 51  6  1 39
 52 42  1 61

In [ ]:
train_len=int(0.9*len(data))
train_data=data[:train_len]
val_data=data[train_len:]
block_size=8
train_data[:block_size+1]

<tf.Tensor: shape=(9,), dtype=int64, numpy=array([18, 47, 56, 57, 58,  1, 15, 47, 58])>

In [ ]:
## Creating X and Y
X=train_data[:block_size]
Y=train_data[:block_size+1]
for t in range (block_size):
  context=X[:t+1]
  target=Y[t]
  print(f"when input is {context} the target: {target}")

when input is [18] the target: 18
when input is [18 47] the target: 47
when input is [18 47 56] the target: 56
when input is [18 47 56 57] the target: 57
when input is [18 47 56 57 58] the target: 58
when input is [18 47 56 57 58  1] the target: 1
when input is [18 47 56 57 58  1 15] the target: 15
when input is [18 47 56 57 58  1 15 47] the target: 47


In [ ]:
tf.random.set_seed(1337)
batch_size=4
block_size=8

def get_batch(split):
  data=train_data if split=='train' else val_data
  ix=tf.random.uniform((batch_size,), minval=0, maxval=len(data)-block_size, dtype=tf.int32)
  x=tf.stack([data[i:i+block_size] for i in ix])
  y=tf.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb,yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb.numpy())
print('targets:')
print(yb.shape)
print(yb.numpy())


inputs:
(4, 8)
[[ 1 51 63  1 41 53 39 58]
 [39 42  0 20 47 57  1 52]
 [32 53  1 56 43 60 43 50]
 [54 39 52 63  1 54 47 43]]
targets:
(4, 8)
[[51 63  1 41 53 39 58  6]
 [42  0 20 47 57  1 52 39]
 [53  1 56 43 60 43 50  1]
 [39 52 63  1 54 47 43 41]]


In [ ]:
class AttentionHeadDecoder(tf.keras.layers.Layer):
  def __init__(self, head_size):
    super(AttentionHeadDecoder, self).__init__()
    self.key=tf.keras.layers.Dense(head_size, use_bias=False)
    self.query=tf.keras.layers.Dense(head_size, use_bias=False)
    self.value=tf.keras.layers.Dense(head_size, use_bias=False)
    self.tril=tf.constant(tf.linalg.band_part(tf.ones(shape=(block_size,block_size)),-1, 0))
    self.dropout=tf.keras.layers.Dropout(dropout)
    self.head_size=head_size

  def call(self, x):
    B,T,C=x.shape
    k=self.key(x)
    q=self.query(x)

    weights=tf.matmul(q, tf.transpose(k, perm=[0, 2, 1])) * tf.math.rsqrt(tf.cast(k.shape[-1], tf.float32))
    weights=tf.where(self.tril[:T, :T]==0,-np.inf, weights) ## This makes it decoder block, since we arent getting the language model.
    weights=tf.nn.softmax(weights, axis=-1)
    weights=self.dropout(weights)
    v=self.value(x)
    output=tf.matmul(weights, v)
    return output

In [ ]:
class MultiHeadAttentionDecoder(tf.keras.layers.Layer):
  def __init__(self, num_heads, head_size):
    super(MultiHeadAttentionDecoder, self).__init__()
    self.heads=[AttentionHeadDecoder(head_size) for _ in range(num_heads)]
    self.projection=tf.keras.layers.Dense(n_embd)
    self.dropout=tf.keras.layers.Dropout(dropout)

  def call(self, x):
    output=tf.concat([h(x) for h in self.heads], axis=-1)
    output=self.projection(output)
    output=self.dropout(output)
    return output

In [ ]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, n_embd):
    super(FeedForward, self).__init__()
    self.ffnet=tf.keras.Sequential([
        tf.keras.layers.Dense(4*n_embd),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Dense(n_embd),
        tf.keras.layers.Dropout(dropout)
    ])
  def call(self, x):
    return self.ffnet(x)

In [ ]:
class Block(tf.keras.layers.Layer):
  def __init__(self, n_embd, n_head):
    super(Block, self).__init__()
    head_size=n_embd//n_head
    self.sa=MultiHeadAttentionDecoder(n_head, head_size)
    self.ff=FeedForward(n_embd)
    self.ln1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.ln2=tf.keras.layers.LayerNormalization(epsilon=1e-6)

  def call(self, x):
    x=x+self.sa(self.ln1(x))
    x=x+self.ff(self.ln2(x))
    return x


In [ ]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.2

In [ ]:
class GptModel(tf.keras.Model):
  def __init__(self, vocab_size):
    super(GptModel, self).__init__()
    self.token_embedding_table=tf.keras.layers.Embedding(vocab_size, n_embd)
    self.position_embedding_table=tf.keras.layers.Embedding(block_size, n_embd)
    self.blocks=tf.keras.Sequential([Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.layerNorm=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.lm_head=tf.keras.layers.Dense(vocab_size, kernel_initializer='normal')

  def call(self, idx, targets=None):
    B,T=idx.shape
    tok_embd=self.token_embedding_table(idx) # (Batch, time, channel) size
    pos_embd=self.position_embedding_table(tf.range(T, dtype=tf.float32)) # (time, channel)
    x=tok_embd+pos_embd #(B,T,C)
    x=self.blocks(x)
    x=self.layerNorm(x)
    logits=self.lm_head(x) #(B,T,vocab_size)

    if targets is None:
      losses2=None
    else:
      B, T,C=logits.shape
      logits=tf.reshape(logits,shape=(B*T,C) )
      targets=tf.reshape(targets, shape=(B*T,))
      losses2=tf.keras.losses.sparse_categorical_crossentropy(targets, logits, from_logits=True)
      #losses2=tf.reduce_mean(losses2)
    return logits, losses2

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_conditionals=idx[:, -block_size:]
      logits, loss=self(idx_conditionals)
      logits=logits[:, -1, :] #becomes( B, C)
      probs=tf.nn.softmax(logits, axis=-1)
      idx_next=tf.random.categorical(tf.math.log(probs), num_samples=1, dtype=tf.int64) #(B,1)
      idx=tf.concat([idx, idx_next],axis=1) # (B, T+1)
    return idx



In [ ]:
gpt_model=GptModel(vocab_size)
# Create a dummy input
dummy_input = tf.zeros((1, block_size), dtype=tf.int32)
# Call the model with the dummy input to build it
_ ,_= gpt_model(dummy_input)
# Now you can access the trainable variables
print(sum(tf.size(v).numpy() for v in gpt_model.trainable_variables)/1e6, 'M parameters')

0.209729 M parameters


In [ ]:
def test_model(model):
  decoder_input = tf.random.uniform(
        (batch_size, block_size),
        minval=0,
        maxval=vocab_size,
        dtype=tf.int32
    )
  decoder_output, loss=model(decoder_input)
  print(f"decoder input-->{decoder_input}, decoder output-->{decoder_output}")
  print(decoder_input.shape,decoder_output.shape)

In [ ]:
test_model(gpt_model)

decoder input-->[[ 1  8  4 60 24 19  5 10 54 29 40 47 15  6 37  6 43 56  1 48 55  1  5 22
  45 31 58 40 18 45 47 57]
 [50  3 22 32 20 36 35 24  7 26  2 39 48 33 20 64 27 52 44 26 54 51 30 32
  28 44 40 23 60 41 26 30]
 [44 52 38  2 19  7  3 47 20 57 54 44 64 21 47 22 19 22  3 28 13 24 31 35
  24 32 11 52 60 40  1 55]
 [13 40 30 50 18 27 26 61 42  2 21 38 64 51 14 50 63 52 62 22 17 63  4 10
  47 33 33 23 48 60 51 21]
 [16 18  5  0 33 62 11  2 40 48 56 57 63 50 27 14 59  3 45 61 19  4  6 25
  64  8 61 35  1 62 64 45]
 [19 10 13 27 59 13 16 20  9  5 61 40  3  8 29 27 45 63 45 23 42 33 52 15
  27 51 23 57 41 45  9 18]
 [40 11 42 57 47 39 30 34 18 32  7 46 36  6 11  2 36 18 13 18 16 46 31 33
  54  1 11 30 18 46 19 27]
 [39 41 13 19 62  5  4 14 12 10 13 39 34 24  1 17  2 43 52 39 15 45 48 16
  37 62 33 24 37 60 31 53]
 [27 30 24  1 33  1 11 51 24 35 16 37 60 20  9 50 21  5 60 18 19 42 38 11
  46  7 35 38 49 36  7 56]
 [39 47 57 20 62 37 53 33 58 34 35 19  3 33  5 19 15  5 62 20 59 39 61 61
 

In [ ]:
#@tf.function
def estimate_loss(model):
    out = {}
    model.trainable=False
    for split in ['train', 'val']:
        losses=tf.TensorArray(tf.float32, size=eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = gpt_model(X, Y)
            losses=losses.write(k, loss)
        out[split] = losses.stack().numpy().mean()
    model.trainable=True
    return out

In [ ]:
optimizer_gpt=tf.keras.optimizers.AdamW(learning_rate=learning_rate)

In [ ]:
#batch_size=32
for steps in range(max_iters):
  if steps%eval_interval==0:
    losses=estimate_loss(gpt_model)
    print(f"step: {steps}, train loss: {losses['train']}, val loss: {losses['val']}")
  xb, yb=get_batch('train')
  with tf.GradientTape() as tape:
    logits2, loss2=gpt_model(xb, yb)
    gradients=tape.gradient(loss2, gpt_model.trainable_variables)
    optimizer_gpt.apply_gradients(zip(gradients, gpt_model.trainable_variables))
#print(loss2.numpy())

step: 0, train loss: 4.307332515716553, val loss: 4.3081955909729
step: 500, train loss: 2.2909929752349854, val loss: 2.297534465789795
step: 1000, train loss: 2.071084976196289, val loss: 2.1373119354248047
step: 1500, train loss: 1.9506285190582275, val loss: 2.0400304794311523
step: 2000, train loss: 1.8504546880722046, val loss: 1.9740591049194336
step: 2500, train loss: 1.7997691631317139, val loss: 1.9491997957229614
step: 3000, train loss: 1.7531301975250244, val loss: 1.9270868301391602
step: 3500, train loss: 1.728566288948059, val loss: 1.8933018445968628
step: 4000, train loss: 1.705342411994934, val loss: 1.8675044775009155
step: 4500, train loss: 1.6775745153427124, val loss: 1.8529760837554932


In [ ]:
context=tf.zeros(shape=(1,1), dtype=tf.int64)
print(decode(gpt_model.generate(context, max_new_tokens=2000)[0].numpy()))


Edwere while follow, 'erm he what you be hast.

DEMINket doss I meak but neadied to fait.

JULIET:
The the spuch: my as their didishing.

Muslove's fatter thus, any my if
Meandun. Boble let move on sad, dest at here yours, come lave by fallow;
Whem look I her speeak agen my doss: spirs,
Misfeel homes what he consweed no pasold as I know gill the regive your will'd you badd handugo now
wo prober ower good I have have as he would peop your honour
And to be the's true in combe the paow an that: and shall you; you grundy!
To for Romemercy tell I low in proisider these will a stave agains!

ESCALUS:
Bair you havy the sayem your rinces,
Is plearelings the effied no pavy me, to lamberacame
Pard he fast: say, chall meang you,
Let leaves it that I kere's spear,
Which is fail enry heaving your clour,
Shot doot, but a why bear'd has giery sand that pospled a the painter of 'best take be' pright daught,
Be a
the excuse his belandsom, he do commpose yore bohander this be twemand's see I me
Let now

## Saving

In [ ]:
## Saving the model weights
gpt_model.save_weights('gpt_model.h5')

# Transformer using tensorflow-keras

#### Positonal encoding in pytorch see it to fix your encoding

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, d_model, max_len=5000):
    super(PositionalEncoding, self).__init__()
    self.d_model = d_model
    self.max_len = max_len
    pe=tf.zeros(shape=(max_len, d_model))
    positions=tf.expand_dims(tf.range(0, max_len, dtype=tf.float32), axis=1)
    div_term=tf.exp(tf.range(0, d_model, 2, dtype=tf.float32) * -(math.log(10000.0) / d_model))
    i_vals=tf.cast(tf.range(d_model), dtype=tf.float32)
    pe=tf.concat([
        [tf.sin(pos/(10000**(2*i/d_model))) if i%2==0 else
        tf.cos(pos/(10000**(2*i/d_model)))
        for i in i_vals]
        for pos in positions
    ], axis=0)
    pe=pe[tf.newaxis, :, :] ### check
    self.pe=tf.Variable(pe, trainable=False)
    self.dropout=tf.keras.layers.Dropout(0.1)

  def call(self, x):
    #op= x+tf.transpose(self.pe[:tf.shape(x)[1], :])
    print(f"shape is-->{ x.shape}, pe shape-> {self.pe[:tf.shape(x)[1], :].shape}")
    op=x+self.pe[tf.newaxis, :tf.shape(x)[1], :]
    return self.dropout(op)


In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, d_model, max_len=5000):
    super(PositionalEncoding, self).__init__()
    self.d_model = d_model
    self.max_len = max_len
    pe=np.zeros(shape=(max_len, d_model), dtype=np.float32)
    positions=tf.expand_dims(tf.range(0, max_len, dtype=tf.float32), axis=1)
    div_term=tf.exp(tf.range(0, d_model, 2, dtype=tf.float32) * -(tf.math.log(10000.0) / d_model))
    i_vals=tf.cast(tf.range(d_model), dtype=tf.float32)
    pe[:, 0::2] = tf.sin(positions * div_term)
    pe[:, 1::2] = tf.cos(positions * div_term)
    self.pe=tf.Variable(pe, trainable=False)
    self.dropout=tf.keras.layers.Dropout(0.1)

  def call(self, x):
    #op= x+tf.transpose(self.pe[:tf.shape(x)[1], :])
    #print(f"shape is-->{ x.shape}, pe shape-> {self.pe[:tf.shape(x)[1], :].shape}")
    #op=x+self.pe[:tf.shape(x)[1], :]
    op=x+tf.expand_dims(self.pe[:tf.shape(x)[1], :], axis=0)
    return self.dropout(op)

In [ ]:
def test_positonal_encoding():
  d_model=64
  max_len=32
  x=tf.random.uniform(shape=(1, max_len), maxval=vocab_size,dtype=tf.int32)
  embed=Embedding(vocab_size, d_model)
  pe=PositionalEncoding(d_model, max_len)
  print(x.shape)
  x=embed(x)
  print(x.shape)
  op=pe(x)
  print(op.shape)
  print(op)

In [ ]:
class Embedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super(Embedding, self).__init__()
    self.embedding=tf.keras.layers.Embedding(vocab_size, d_model)

  def call(self, x):
    return self.embedding(x)

In [ ]:
test_positonal_encoding()

(1, 32)
(1, 32, 64)
(1, 32, 64)
tf.Tensor(
[[[ 2.9965367e-02  9.7622323e-01 -6.2834099e-04 ...  9.5219421e-01
    1.1789013e-02  1.0149907e+00]
  [ 8.5415262e-01  5.3590965e-01  7.2832125e-01 ...  1.0037460e+00
   -1.4083704e-03  9.6254164e-01]
  [ 9.1822082e-01 -4.2171818e-01  1.0372987e+00 ...  9.9012125e-01
   -3.3736184e-02  1.0192972e+00]
  ...
  [-6.3590962e-01 -7.5702393e-01  2.1274668e-01 ...  1.0133574e+00
   -1.1203975e-02  9.8147833e-01]
  [-9.7006452e-01  1.7915165e-01 -4.9107462e-01 ...  1.0339257e+00
   -4.0074687e-02  9.7387457e-01]
  [-3.8931236e-01  9.1204524e-01 -9.5500493e-01 ...  1.0410236e+00
   -9.0732351e-03  1.0373031e+00]]], shape=(1, 32, 64), dtype=float32)


In [ ]:
def positional_encoding(length, depth):
    # Use TensorFlow operations instead of NumPy
    depth = depth // 2
    positions = tf.range(length)[:, tf.newaxis]
    depths = tf.range(depth)[tf.newaxis, :] / depth
    angle_rates = 1 / (10000 ** tf.cast(depths, tf.float32))
    angle_rads = tf.cast(positions, tf.float32) * angle_rates
    pos_encoding = tf.concat([tf.sin(angle_rads), tf.cos(angle_rads)], axis=-1)
    return pos_encoding


In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model, length=5000):
    super(PositionalEmbedding, self).__init__()
    self.d_model=d_model
    self.embedding=tf.keras.layers.Embedding(vocab_size, d_model)
    self.pos_encoding=positional_encoding(length=length, depth=d_model)

  def call(self, x):
    length=tf.shape(x)[-1]
    x=self.embedding(x)
    x*=tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x=x+self.pos_encoding[tf.newaxis,
                          :length, :]
    return x

In [ ]:
def test_positional_embedding():
  d_model=64
  max_len=32
  x=tf.random.uniform(shape=(1, max_len), maxval=vocab_size,dtype=tf.int32)
 # embed=Embedding(vocab_size, d_model)
  pe=PositionalEmbedding(vocab_size, d_model, max_len)
  print(x.shape)
  op=pe(x)
  print(op.shape)

In [ ]:
test_positional_embedding()

(1, 32)
(1, 32, 64)


In [ ]:
class BaseAttentionLayer(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha=tf.keras.layers.MultiHeadAttention(**kwargs, use_bias=False)
    self.layernorm=tf.keras.layers.LayerNormalization()
    self.add=tf.keras.layers.Add()
    self.dropout=tf.keras.layers.Dropout(0.1)

In [ ]:
class CausalSelfAttention(BaseAttentionLayer):
  def call(self, x):
    attn_output=self.mha(query=x, value=x, key=x, use_causal_mask=True)
    x=self.add([x, attn_output])
    x=self.layernorm(x)
    x=self.dropout(x)
    return x

In [ ]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff):
    super(FeedForward, self).__init__()
    self.ffnet=tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model),
        tf.keras.layers.Dropout(0.1)
    ])
    self.layernorm=tf.keras.layers.LayerNormalization()
    self.add=tf.keras.layers.Add()

  def call(self, x):
    ff_output=self.ffnet(x)
    x=self.add([x, ff_output])
    x=self.layernorm(x)
    return x

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, dropout=0.1):
    super(DecoderLayer, self).__init__()
    self.causal_attention=CausalSelfAttention(num_heads=num_heads, key_dim=d_model, dropout=dropout)
    self.feed_forward=FeedForward(d_model, dff)

  def call(self, x):
    x=self.causal_attention(x)
    x=self.feed_forward(x)
    return x

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, dropout=0.1):
    super(Decoder, self).__init__()
    self.d_model=d_model
    self.num_layers=num_layers

    #self.embedding=Embedding(vocab_size, d_model)
    #self.positional_encoding=tf.keras.layers.Embedding(block_size, d_model) # this is just for checking
    #self.pos_encoding=PositionalEncoding(d_model)
    self.positional_embedding=PositionalEmbedding(vocab_size, d_model)
    self.dec_layers=[DecoderLayer(d_model, num_heads, dff, dropout) for _ in range(num_layers)]
    self.final_layer=tf.keras.layers.Dense(vocab_size, kernel_initializer='normal')

  def call(self,x ):
    #x=self.embedding(x)
    #x=x+self.positional_encoding(x)
    x=self.positional_embedding(x)
    for i in range(self.num_layers):
      x=self.dec_layers[i](x)
    output=self.final_layer(x)
    return output


In [ ]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 500
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.2

### Dummy AI based debugging model

In [ ]:
import tensorflow as tf
import numpy as np

def positional_encoding(length, depth):
    # Use TensorFlow operations instead of NumPy
    depth = depth // 2
    positions = tf.range(length)[:, tf.newaxis]
    depths = tf.range(depth)[tf.newaxis, :] / depth
    angle_rates = 1 / (10000 ** tf.cast(depths, tf.float32))
    angle_rads = tf.cast(positions, tf.float32) * angle_rates
    pos_encoding = tf.concat([tf.sin(angle_rads), tf.cos(angle_rads)], axis=-1)
    return pos_encoding

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model, length=5000):
        super(PositionalEmbedding, self).__init__()
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model)
        self.pos_encoding = positional_encoding(length=length, depth=d_model)

    def call(self, x):
        length = tf.shape(x)[-1]
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[:length, :]
        return x

class BaseAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs, use_bias=False)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()
        self.dropout = tf.keras.layers.Dropout(0.1)

class CausalSelfAttention(BaseAttentionLayer):
    def call(self, x):
        attn_output = self.mha(query=x, value=x, key=x, use_causal_mask=True)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        x = self.dropout(x)
        return x

class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff):
        super(FeedForward, self).__init__()
        self.ffnet = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(0.1)
        ])
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

    def call(self, x):
        ff_output = self.ffnet(x)
        x = self.add([x, ff_output])
        x = self.layernorm(x)
        return x

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.causal_attention = CausalSelfAttention(num_heads=num_heads, key_dim=d_model, dropout=dropout)
        self.feed_forward = FeedForward(d_model, dff)

    def call(self, x):
        x = self.causal_attention(x)
        x = self.feed_forward(x)
        return x

class Decoder(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, dropout=0.1):
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.positional_embedding = PositionalEmbedding(vocab_size, d_model)
        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, dropout) for _ in range(num_layers)]
        self.final_layer = tf.keras.layers.Dense(vocab_size, kernel_initializer='normal')

    def call(self, x):
        x = self.positional_embedding(x)
        for i in range(self.num_layers):
            x = self.dec_layers[i](x)
        output = self.final_layer(x)
        return output

def test_positional_embedding():
    d_model = 64
    max_len = 32
    vocab_size = 10000  # Define vocab_size
    x = tf.random.uniform(shape=(1, max_len), maxval=vocab_size, dtype=tf.int32)
    pe = PositionalEmbedding(vocab_size, d_model, max_len)
    print(x.shape)
    op = pe(x)
    print(op.shape)

# Run the test
test_positional_embedding()


(1, 32)
(1, 32, 64)


In [ ]:
decoder_model=Decoder(num_layers=n_layer, d_model=n_embd, num_heads=n_head, dff=4*n_embd, vocab_size=vocab_size)

#### Loss function for production_decoder

In [ ]:
def loss_function(labels, logits):
  return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True))


In [ ]:
#@tf.function
def estimate_loss(model):
    out = {}
    model.trainable=False
    for split in ['train', 'val']:
        losses=tf.TensorArray(tf.float32, size=eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits=model(X)
            loss2=loss_function(Y, logits)
            losses=losses.write(k, loss2)
        out[split] = losses.stack().numpy().mean()
    model.trainable=True
    return out

In [ ]:
out=estimate_loss(decoder_model)
print(out)

KeyboardInterrupt: 

#### Training loop for the model

In [ ]:
## Training loop
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
for steps in range(max_iters):
  xb, yb=get_batch('train')
  with tf.GradientTape() as tape:
    logits=decoder_model(xb)
    loss=loss_function(yb, logits)
    print(f"loss->{ loss}, count-->{steps}")
    gradients=tape.gradient(loss, decoder_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, decoder_model.trainable_variables))


loss->4.326534271240234, count-->0


ValueError: not enough values to unpack (expected 2, got 0)

## Debugging

In [ ]:
# Before computing gradients
print("Before Gradient Calculation:")
for var in decoder_model.trainable_variables:
    print(var.name)
xb, yb=get_batch('train')
print(xb.shape, yb.shape)
with tf.GradientTape() as tape:
    logits = decoder_model(xb)
    loss = loss_function(yb, logits)

# After computing gradients
print("After Gradient Calculation:")
for var in decoder_model.trainable_variables:
    print(var.name)


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3578, in run_code
    return outflag  File "<ipython-input-89-f3f86f71cc0c>", line 6, in <cell line: 0>
    losses=estimate_loss(decoder_model)  File "<ipython-input-82-b6d0757c28bf>", line 8, in estimate_loss
    X, Y = get_batch(split)  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


Before Gradient Calculation:
(16, 32) (16, 32)
After Gradient Calculation:


In [ ]:
decoder_model.compile(optimizer='adamw', loss='sparse_categorical_crossentropy')


In [ ]:
for layer in decoder_model.layers:
    print(f"{layer.name}: trainable={layer.trainable}")

positional_embedding_7: trainable=True
decoder_layer_12: trainable=True
decoder_layer_13: trainable=True
decoder_layer_14: trainable=True
decoder_layer_15: trainable=True
dense_35: trainable=True


In [ ]:
for layer in decoder_model.layers:
    layer.trainable = True

In [ ]:
for layer in decoder_model.layers:
    print(f"{layer.name}: trainable={layer.trainable}")


embedding_8: trainable=True
positional_embedding_5: trainable=True
decoder_layer_8: trainable=True
decoder_layer_9: trainable=True
decoder_layer_10: trainable=True
decoder_layer_11: trainable=True
dense_26: trainable=True


In [ ]:
for var in decoder_model.trainable_variables:
    print(var.name)


embeddings
kernel
kernel
kernel
kernel
gamma
beta
kernel
bias
kernel
bias
gamma
beta
kernel
kernel
kernel
kernel
gamma
beta
kernel
bias
kernel
bias
gamma
beta
kernel
kernel
kernel
kernel
gamma
beta
kernel
bias
kernel
bias
gamma
beta
kernel
kernel
kernel
kernel
gamma
beta
kernel
bias
kernel
bias
gamma
beta
kernel
bias


In [ ]:
dummy_input = tf.constant([[1, 32]])  # Replace with appropriate input shape
decoder_model(dummy_input)

<tf.Tensor: shape=(1, 2, 65), dtype=float32, numpy=
array([[[ 0.20314631,  0.08497136, -0.8745965 ,  0.11160021,
         -0.1377081 , -0.24788046,  0.21647897,  0.5213624 ,
          0.29480776, -0.64917105,  0.18583238, -0.37512958,
         -0.0262249 ,  0.01239629,  0.93078107, -0.58524305,
          0.09809493, -0.47205967, -0.18520868,  0.09592154,
         -0.49041814, -0.21090993, -0.16654561,  0.3900448 ,
         -0.2523714 ,  0.28446776,  0.37590474,  0.29830053,
          0.04083604, -0.23921514,  0.47719157, -0.16996107,
          0.07034269,  0.02197703,  0.12283357,  0.11808436,
          0.20611233,  0.17101255, -0.13200939, -0.13331053,
          0.13868141, -0.19393566, -0.9601563 ,  0.49952084,
         -0.02267453,  0.08059627,  0.56349736, -0.24792361,
         -0.4541805 , -0.18882255, -0.67917126, -0.0900622 ,
         -0.04652077,  0.17453033,  0.4004238 ,  0.8158908 ,
         -0.00450779, -0.34449935,  0.38634402, -0.40474808,
          0.15729235,  0.33964124

# Training Loop

In [ ]:
## Training loop with loss estimate
optimizer=tf.keras.optimizers.AdamW(learning_rate=learning_rate)
for steps in range(max_iters):
  xb, yb=get_batch('train')
  if steps%eval_interval==0:
    losses=estimate_loss(decoder_model)
    print(f"step: {steps}, train loss: {losses['train']}, val loss: {losses['val']}")
  with tf.GradientTape() as tape:
    logits=decoder_model(xb)
    #print(xb.shape, yb.shape, logits.shape)
    loss=loss_function(yb, logits)
    gradients=tape.gradient(loss, decoder_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, decoder_model.trainable_variables))


step: 0, train loss: 4.250707149505615, val loss: 4.2519211769104
step: 500, train loss: 2.3494510650634766, val loss: 2.358513116836548
step: 1000, train loss: 2.1575987339019775, val loss: 2.180633783340454
step: 1500, train loss: 2.0135529041290283, val loss: 2.0672426223754883
step: 2000, train loss: 1.9311662912368774, val loss: 2.0035011768341064
step: 2500, train loss: 1.8728077411651611, val loss: 1.9795480966567993
step: 3000, train loss: 1.8267226219177246, val loss: 1.9378784894943237
step: 3500, train loss: 1.7934709787368774, val loss: 1.9148969650268555
step: 4000, train loss: 1.7521355152130127, val loss: 1.9034076929092407
step: 4500, train loss: 1.7239738702774048, val loss: 1.870065450668335


In [ ]:
def generator(decoder_model, idx, max_new_tokens):
  for _ in range(max_new_tokens):
    idx_conditionals=idx[:, -block_size:]
    logits=decoder_model(idx_conditionals)
    logits=logits[:, -1, :]
    probs=tf.nn.softmax(logits, axis=-1)
    idx_next=tf.random.categorical(tf.math.log(probs), num_samples=1, dtype=tf.int64)
    idx=tf.concat([idx, idx_next],axis=1)
  return idx


In [ ]:
print(decode(generator(decoder_model, tf.zeros(shape=(1,1), dtype=tf.int64), max_new_tokens=2000)[0].numpy()))


And bainsts.

RABELAM:
Praliese more gobus grow of his good geent,
In my bruch heer chear track his feid combervoak!

MENENIUS:
This dispose!

EdEtward I my save was, with one,
welce, you are contry 'to nie; eneese weret worthy roth withom,
I som thanstrious men which as weed,
I was good see die; and to dey
I done us some that would, and hope shaln tear's her's 'four of
meved the shorey, fierss her, dispalty,
It for for comblen you, and b sod.
To daid you see; or there's greal ain was byseet sweet,
I poy patcitione revish-your a greemiter.

WERWAW:
Have so, sir, mones his one for olling you,
but yough hath onor tain?

MENENIUS:
I gray her not one as,
comper wortwoms eyencesssiffor so!
And fear geneat of jest, feat:
To you weseel.

LADY NENV:
Have no
O't:
Yesee, sur, Is ame to your you word:
Consephind to marre so with once complo
orbanis worctcown soack her cound
To may fair onote: were unknow's not of
Where, no dears; we my shall, shet pirise give as owdering that full'd time
Hath sp

In [ ]:
## Save decoder_model
decoder_model.save('decoder_model.keras')


In [ ]:
def test_decoder_model(decoder_model):
  decoder_input = tf.random.uniform(
        (batch_size, block_size),
        minval=0,
        maxval=vocab_size,
        dtype=tf.int32
    )
  decoder_output=decoder_model(decoder_input)
  print(f"decoder input-->{decoder_input}, decoder output-->{decoder_output}")
  print(decoder_input.shape,decoder_output.shape)

In [ ]:
test_decoder_model(decoder_model)

decoder input-->[[18 61 22 10 39 45 40 29 53 61  2 34  3 37  2 30 58  5 39 56 59 30 16 32
  48 54  7 33 54 18 57 29]
 [62 26 47 30 38 21 12  8 27 50 14 53 42 59 40 49 14 60 31 58 58 30  0 52
  16 22 43 28 28 49 21 60]
 [14 16 33 26 59 18 54 27 53  4 16 44 45  7 30  0 31 60 15 48 20 34  6 27
  28 44 23 23 29 11 45 34]
 [59 16 11  2  4 58 15 19  4 60 64 33 13 10 52 30 60 43  2 48 31 58 59 27
   5 60 32 64 55 20 28 29]
 [13 64 10 12 11 51 56 60 52 48  6 25 47 26 23 10 63 27  7 62 53 46 10 15
  24 31 38 52 27 50 23 43]
 [ 7 29 12 60 33 61 45 40 34  8 25 19 14 29 64 14 31 35 61 11 46 13 26 34
  44 47 25 17 27 15 36 17]
 [34 46 61 42 54 37 49 60 52 28 26  4 25 18 30 23 20 33  6 57 37 59 24 23
  57  8 10  3 54 62  5 19]
 [37 55 41 45 62 20 34 42 18 39 59 32 32 10 25 62  7 11 61 59  2 45 23 48
  13 41  8  4  9 33 24 46]
 [17 59  7 40 23 50 23 36 19 17 47 64  8 20 11 55 54 12 61 21 34 16 29 60
  15 60 53  5 56 53 22 40]
 [35 17 38 57 40 40 35 53  1 49 38 55 59 56 62  4 24 26 62 49 60 51 44 25
 